# RLMTP Basic Example

rlmtp contains tools to post-process material coupon test data. This notebook contains shows the basic usage of this package. The first step is to load the package.

In [1]:
import rlmtp

The rest of this example illustrates the functionality of rlmtp. First, let's load some other useful packages for this example.

In [2]:
from IPython.display import Image, IFrame

## Sync stress-strain data with temperature data

Since we record the stress-strain and temperature through two different softwares, we would like to syncronize the measurements. This is done by matching the system times between the two sets of recordings --- rlmtp takes care of this step for you. First, we need to import the data from our different sources. Two functions are provided for this task, make sure that the data files follow the established format, otherwise the import will fail.

In [3]:
path_to_dion_data = '../rlmtp/tests/LP9_1_181211.xlsx'  # replace with your path
dion_data = rlmtp.import_dion7_data(path_to_dion_data)
path_to_catman_data = '../rlmtp/tests/Temp_LP9_1_181211.XLSX'  # replace with your path
catman_data = rlmtp.import_catman_data(path_to_catman_data)

Now we can simply syncronize the data, and add the temperature measurements to the stress-strain measurements. The syncrhonization is done based on the 'System Date' column that exists within dion_data.data and catman_data.data. A copy of the Dion7 data is returned with a column of 'Temperature[C]'.

In [4]:
synced_data = rlmtp.sync_temperature(dion_data, catman_data)

The measurement data is now accessed through synced_data. The returned data is a pandas.DataFrame object.

## Process databases stored in the specified format

A protocol is provided for the storage of coupon data, your life is made much easier if you follow this format. In this example the data is loaded automatically, the temperature is synchronized, the synchronized data is output, and plots are generated. In this example, the coupon data stored in the "test_specimen" directory follows our format.

In [5]:
coupon_dir = '../rlmtp/tests/test_specimen/'
output_dir = './'
rlmtp.process_specimen_data(coupon_dir, output_dir)

Checking files...
	 Dion7 data exists.
	 catman data exists.
	 Filtering information does NOT exist.
Syncing temperature data with Dion7 data.


The function let's you know what data it finds, if the data is not found then: 1) it does not exist, 2) it is not named properly, or 3) it is not in the correct location. The process_specimen_data() function is useful to easily process entire databases by creating a loop over all the directories that contain coupon data.

The output from process_specimen_data() is generated in [output_dir]/[coupon_root], where coupon_root is the name of the directory that contains all the data for the coupon test. In this example the output is in "./test_specimen/", you can see: processed_data.csv, containing the test data; stress_strain_plot.pdf, the true stress - true strain plot; and temperature_strain_plot.pdf, the temperature plotted versus true strain. Let's look at the temperature response:

In [7]:
IFrame("./test_specimen/temperature_strain_plot.pdf", width=400, height=300)

From this plot we can see that the temperature rise is negligible for this specimen.

## Capture images at specified time steps

VERY IMPORTANT: This functionality requires that ffmpeg is installed (equivalently, ffmpeg and ffprobe are on the Path) on your system. ffmpeg is a free software available at: https://www.ffmpeg.org/ .

We may be interested to correlate the deformed shape of a specimen with particular time increments, for example to see if buckling has occured. For this example we need to load a new data set, and make sure we have the path to the video file of interest.

In [ ]:
path_to_dion_data_2 = '../rlmtp/tests/testData_07012018.xlsx'
dion_data_2 = rlmtp.import_dion7_data(path_to_dion_data_2)
video = '../rlmtp/tests/D5600_B_254_132139_589.MOV'  # this file may not exist for you since it's around 1Gb we don't include it

For the following commands to work, rlmtp requires that the video starts before the Dion7 recordings, and ends after the Dion7 recordings. Therefore, take care when you are starting and stopping the video. In this example, we are interested in seeing if buckling has occured at the excursion to -5.5 % strain. We identified this as the 18059'th time increment. First, we get the time in the Dion7 data corresponding to this increment, then find how many seconds into the video this time occurs.

In [ ]:
time_55_strain = dion_data_2.get_times_at_increments([18059])
video_time = rlmtp.dion7_times_to_video_times(video, time_55_strain)

In the above command, multiple increments can be added to the list (in ascending order) to get multiple times, however we just wanted one picture. Now we use the time from Dion7 to capture the frames from the video. The pictures are saved in jpeg format to output_dir, with the format [file\_name]\_[S]\_[ms].jpg, where S is the seconds in the video, and ms are the milliseconds.

In [ ]:
output_dir = './'
file_name = 'ex_1_pic'
rlmtp.output_frames_at_times(video, video_time, output_dir, file_name)

Now you can load the image and see the results.

In [ ]:
Image("./ex_1_pic_916_394.jpg")

Clearly buckling has occured at this stage!